In [21]:
using DynamicPolynomials, SparseArrays, MatrixMarket
println("***Problem setting***")



# random quadratic objective function f
data="/home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/dataPOP2" # path of data
A = MatrixMarket.mmread(data*"/GD95_a.mtx")
n=size(A,1)
@polyvar x[1:n]# variables

println("Number of variable: n=",n)

f=-1.0*sum(A[i,j]*x[i]*(1-x[j]) for i=1:n for j=1:n)


# unit sphere constraint
m=0#ceil(Int64, n/5)
g=Vector{Polynomial{true,Float64}}(undef,m)
 #type of coefficients of each polynomial must be float

randx=rand(n)# create a feasible solution
for j in 1:n
    if randx[j]>0.5
        randx[j]=1
    else
        randx[j]=0
    end
end
for j in 1:m
    g[j]=generate_random_poly(v[2:end])
    g[j]+=0.125-g[j](x => randx) #make constraints feasible
end



m=length(g)
println("Number of inequality constraints: m=",m)

l=0#ceil(Int64,n/5)

h=Vector{Polynomial{true,Float64}}(undef,l)
for j in 1:l
    h[j]=generate_random_poly(v[2:end])
    h[j]-=h[j](x => randx) #make constraints feasible
end

h=[h;[x[j]*(1-x[j]) for j=1:n]]

l=length(h)
println("Number of equality constraints: l=",l)

***Problem setting***
Number of variable: n=36
Number of inequality constraints: m=0
Number of equality constraints: l=36


In [22]:
Matrix(A)

36×36 Array{Bool,2}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  1  0
 1  0  0  0  0  1  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  1  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  1  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  1  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     

In [19]:
include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=false);

k=0
s=30

@time opt_val1,opt_sol=InterRelax.RelaxDense(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,
    lmon_f,supp_f,coe_f,dg,dh,k,s,solver="Mosek",comp_opt_sol=false);

Relaxation order: k=0
Block size: s=30


Maximal matrix size:29
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 435             
  Cones                  : 0               
  Scalar variables       : 435             
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   :

In [20]:
using SparseArrays

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);
#include("./densePOPvar$(n)nineq$(m)neq$(l).jl");

using TSSOS

k=1

println("Maximal matrix size: ",binomial(n+k,k))

vars,pop=InterRelax.get_POP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)

@time opt,sol,data=TSSOS.cs_tssos_first(pop,vars,k,numeq=l,CS=false,TS=false,solver="Mosek");

Maximal matrix size: 29
************************TSSOS************************


TSSOS is launching...
------------------------------------------------------
The clique sizes of varibles:
[28]
[1]
------------------------------------------------------
Assembling the SDP...
There are 435 affine constraints.
SDP assembling time: 0.001543814 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 435             
  Cones                  : 0               
  Scalar variables       : 57              
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1      